# Project 3: Understanding User Behavior
**Project Team: Jude Wentian Zhu, Rohit Barkshi, Rathin Bector**

## Overview of Project Files

4 Files:
- project_3.ipynb
    - contains the description of the project, game, game events, shell script, data pipeline. It also contains all the Python Code and shell commands used in the process of the data pipeline from sourcing the data, injesting it, storing it and then analyzing it. This file also contains the business questions and the analysis done in SQL with Presto.
- game_api.py
    - contains the game with the Flask API that generates events.
- docker-compose.yml
    - contains the configurations for setting up the cluster with all of the required containers and network specifications for zookeeper, kafka, cloudera hadoop, spark, presto, redis and mids.
- data-generator.sh
    - contains the shell script used to generate randomized events with Apache Bench

## Project Goal

In this project we stream and store events generated from a game in order to answer some important business questions. In our data pipeline we use new age data engineering tools such as Kafka, Spark, HDFS, Hive and Presto.
For the bonus, we also use Redis to store track player states in the game.

## Explanation of Game (game_api.py)

Our game is a simple multiplayer fighting game. Each player initializes with a set amount of health, money, and a basic weapon (stick). They can purchase another weapon with their money or purchase a shield to limit their damage intake. To get more money, they can "dig for gold". A player attacks other players with their weapon with the hope of killing the other players (bringing their health to 0). If they are successful in killing another player, they inherit all of the other player's money.

### Explanation of function calls:
- /initialize
    - Person calls this to initialize a new player in the game with a given username. The player is initialized with 100 money, 20 health, a stick (weapon) and no shield.
    - The event is logged to Kafka.
- /purchase_weapon
    - Person calls this to purchase a weapon in the game with their given username and desired weapon. There are checks for validity of username and weapon, aliveness of player, and sufficiency of money. If purchase is successful, the appropriate money is subtracted from the wallet, and the new weapon is stored for the player.
    - Types of Weapon: Price, Damage, Success Rate of Weapon
        - Stick: 0, 0, 1
        - Knife: 5, 2, 0.7
        - Sword: 10, 3, 0.65
        - Grenade: 15, 5, 0.4
        - Gun: 50, 9, 0.75
        - Bazooka: 100, 20, 0.5
        - Nuke: 500, 100, 1
    - A successful purchase is logged to Kafka.
- /purchase_shield
    - Person calls this to purchase a shield in the game. There are checks for validity of username, aliveness of player, and sufficiency of money. If purchase is successful, the appropriate money is subtracted from the wallet, and the player is equipped with a shield.
     - Shield costs 20 and limits the damage of attacks by 5.
     - A successful purchase is logged to Kafka.
- /dig_for_gold
    - Person calls this to "dig for gold" or get more money. There are checks for validity of username, aliveness of player, and sufficiency of money to purchase a shovel to "dig for gold". If purchase is successful, the player digs for gold. A random distribution determines the amount of gold the player will receive, and this amount is added to the player's wallet. Digging for gold costs 5.
     - A successful "dig for gold" event is logged to Kafka.
- /attack
    - Person calls this to attack another player. There are checks for validity of both usernames and aliveness of players. The attack has a chance of success based on the player's weapon. If the attack is successful, the enemy player's health is reduced by the player's weapon's damage rate. If the enemy has a shield, the damage rate is reduced by 5 health. The enemy player is killed if their health reaches 0. If the enemy is killed, the player inherits all of the enemy's money in their wallet.
    - A successful attack is logged to Kafka.
    - A failed attack is logged to Kafka.

### Bonus (Redis)
We use redis to track the player state and store their data in the game. We use redis as a distributed, in memory key-value storage engine and populate it with different types of keys and values.

## Explanation of Shell Script

The shell script is run to generate a large number of events calling functions defined in game_api. It's set to run 1000.

** For each loop **

- a random USERID and a random ENEMYID is generated, for example "user14". The script bounds the range of USERIDS and ENEMYIDs to between "user1" to "user50"

- a weapon is selected from a pre-defined set of weapons [Stick, Knife, Sword, Grenade, Gun, Bazooka, Nuke]. This weapon can be be called for purchase or to attack

- if the generated USERID and ENEMYID happen to be the same, another ENEMYID will be generated until these two IDs are distinctive

- In the 1st loop, one user and one enemy are initialized to start the game

- one of the following actions can be taken by the user based on the random value stored in $STAGE variable

     - 11% probability an enemy will be added
     - 22% probability a weapon can be purchased. Although game_api determines whether the user has sufficient fund in wallet to purchase the weapon
     - 11% probability a shield is purchased, if the user has enough fund
     - 45% probability the user will attack the enemy generated at the begging of the loop. We want to increase the frequency of this event because this leads to more interesting analysis.
     - 11% probability the user digs for gold 5 times

Once the maximum number of requests is reached (in our case, 1000 times), the generation ends

## Explanation of Data Pipeline
Here we describe the data pipeline represented by our project starting from data generation at the source to injesting it, to processing it, to storing it, and finally to analyzing it.

### game_api.py logs events to Kafka
The game logs events for at least each of the function calls to Kafka. We utilize the Flask API to log these events as JSONs into the Kafka cluster with an "events" topic where the events are published. 

### Event Generation
We use a shell script with Apache Bench that create a maximum of 50 users in the game with 1000 request call to the game API. The parameters for the requests are randomized as well as the event requests themselves with a set probability. Apache Bench is used in the shell script to execute these randomized requests to generate game events.

### Event Consumption
We consume and process the raw events with Spark (pyspark) where we stream the events, read them and store them in a Spark dataframe. We then filter the raw events into classified events (weapon_purchase events, gold dig events etc.) with predefined schema into Spark dataframes.

### Event Storage
After filtering the extracted events into their respective dataframes, we write them to HDFS as Parquet files.

### Read from HDFS into Memory and Publish Hive Tables
We read from the Parquet files in HDFS using Spark dataframes and publish the data into external tables (schema-on-read) using Hive.

### Presto Analysis
We query the external tables with Presto for analysis of the events in the tables. We answer the business analysis questions in this final step.

## Data Pipeline Steps

### Step 1: Spin up Docker-Compose and Link Pypark to Jupyter Notebook

    1. Spin up the cluster
```console
docker-compose up -d
```

```console
Creating network "project_3_default" with the default driver
Creating project_3_redis_1     ... done
Creating project_3_presto_1    ... done
Creating project_3_mids_1      ... done
Creating project_3_cloudera_1  ... done
Creating project_3_zookeeper_1 ... done
Creating project_3_spark_1     ... done
Creating project_3_kafka_1     ... done
```

    2. Exec a bash shell in the spark container on teminal.
```console
docker-compose exec spark bash
```


    3. Create a symbolic link from the spark directory to /w205 :
```console
ln -s /w205 w205
```


    4. Exit the container
```console
exit
```


    5. Check out Hadoop
```console
docker-compose exec cloudera hadoop fs -ls /tmp/
```

```console
Found 3 items
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2021-08-05 04:36 /tmp/hive
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs
```

### Step 2: Launch Kafka and Flask

    1. Create a kafka topic called events
```console
docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```
```
Created topic events.
```


    2. Install dependencies for flask app on mids container
```console
docker-compose exec mids pip install redis
docker-compose exec mids pip install numpy==1.14.6
```


    3. Start up game_api.py using flask on mids container
```console
docker-compose exec mids env FLASK_APP=/w205/project_3/game_api.py flask run --host 0.0.0.0
```


    4. Start a Jupyter notebook for a pyspark kernal
```console
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
```

### Step 3: Spark Streaming Setup
#### 3.1 Import Packages

In [1]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType

#### 3.2 Define Event Schema 

In [2]:
def player_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
    ])

def purchase_weapon_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- weapon: string (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("weapon", StringType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
    ])

def purchase_shield_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
    ])

def dig_for_gold_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- gold_found: integer (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("gold_found", IntegerType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
    ])

def successful_attack_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- attacker: string (nullable = true)
    |-- defender: string (nullable = true)
    |-- weapon_used: string (nullable = true)
    |-- defender_has_shield: boolean (nullable = true)
    |-- defender_health_before:integer  (nullable = true)
    |-- defender_health_after:integer  (nullable = true)
    |-- defender_killed: boolean (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("attacker", StringType(), True),
        StructField("defender", StringType(), True),
        StructField("weapon_used", StringType(), True),
        StructField("defender_has_shield", BooleanType(), True),
        StructField("defender_health_before", IntegerType(), True),
        StructField("defender_health_after", IntegerType(), True),
        StructField("defender_killed", BooleanType(), True),
    ])

def failed_attack_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- attacker: string (nullable = true)
    |-- defender: string (nullable = true)
    |-- weapon_used: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("attacker", StringType(), True),
        StructField("defender", StringType(), True),
        StructField("weapon_used", StringType(), True),
    ])

#### 3.3 Define Event Filtering Functions

In [3]:
@udf('boolean')
def is_player(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'initialize_player':
        return True
    return False


@udf('boolean')
def is_purchase_weapon(event_as_json):
    event = json.loads(event_as_json)
    # m = re.match('purchase',event['event_type'])
    if event['event_type'] == 'purchase_weapon':
        return True
    return False

@udf('boolean')
def is_purchase_shield(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_shield':
        return True
    return False


@udf('boolean')
def is_dig_for_gold(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'dig_for_gold':
        return True
    return False


@udf('boolean')
def is_purchase_shield(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_shield':
        return True
    return False

@udf('boolean')
def is_successful_attack(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'successful_attack':
        return True
    return False

@udf('boolean')
def is_failed_attack(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'failed_attack':
        return True
    return False


#### 3.4 Start Stream Read

In [4]:
### Start Spark Session
spark = SparkSession \
    .builder \
    .appName("ExtractEventsJob") \
    .getOrCreate()

### Subscribe to Kafka events topic for streaming
raw_events = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .load()

#### 3.5 Extract Events and Write to HDFS

In [5]:
#extract initial_player events
extracted_initialize_player = raw_events \
    .filter(is_player(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      player_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')

sink_player = extracted_initialize_player \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/player") \
    .option("checkpointLocation", "/tmp/checkpoint_player") \
    .start()
    
    
#extract purachase_weapon events
extracted_purchase_weapon = raw_events \
    .filter(is_purchase_weapon(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_weapon_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_purchase_weapon = extracted_purchase_weapon \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/purchase_weapon") \
    .option("checkpointLocation", "/tmp/checkpoint_purchase_weapon") \
    .start()

    
#extract purachase_shield events
extracted_purchase_shield = raw_events \
    .filter(is_purchase_shield(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_shield_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
            
sink_purchase_shield = extracted_purchase_shield \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/purchase_shield") \
    .option("checkpointLocation", "/tmp/checkpoint_purchase_shield") \
    .start()


#extract dig for gold events
extracted_dig_for_gold = raw_events \
    .filter(is_dig_for_gold(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                  dig_for_gold_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_dig_for_gold = extracted_dig_for_gold \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/dig_for_gold") \
    .option("checkpointLocation", "/tmp/checkpoint_dig_for_gold") \
    .start()    

#extract successful_attack events
extracted_successful_attack = raw_events \
    .filter(is_successful_attack(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      successful_attack_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_successful_attack= extracted_successful_attack \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/successful_attack") \
    .option("checkpointLocation", "/tmp/checkpoint_successful_attack") \
    .start()
            
            
#extract failed_attack events
extracted_failed_attack = raw_events \
    .filter(is_failed_attack(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      failed_attack_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_failed_attack= extracted_failed_attack \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/failed_attack") \
    .option("checkpointLocation", "/tmp/checkpoint_failed_attack") \
    .start()

#### 3.5 Generate events using `data_generator.sh` bash script
We generate events using a shell script that uses Apache Bench to send a stream of randomized requests.

```console
bash data_generator.sh
```

#### 3.6 Check Events in Kafka 

```console
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e
```

Output exerpt in Kafka
```
{"event_type": "successful_attack", "weapon_used": "Knife", "defender_killed": false, "attacker": "user30", "defende
r_health_before": 2, "defender": "user32", "defender_health_after": 2, "Accept": "*/*", "User-Agent": "ApacheBench/2
.3", "Host": "user1.comcast.com", "defender_has_shield": true}
{"username": "user24", "event_type": "purchase_weapon", "weapon": "Bazooka", "Accept": "*/*", "User-Agent": "ApacheB
ench/2.3", "wallet_after": 113, "Host": "user1.comcast.com", "wallet_before": 213}
{"event_type": "successful_attack", "weapon_used": "Bazooka", "defender_killed": true, "attacker": "user8", "defende
r_health_before": 20, "defender": "user50", "defender_health_after": 0, "Accept": "*/*", "User-Agent": "ApacheBench/
2.3", "Host": "user1.comcast.com", "defender_has_shield": false}

```

#### 3.7 Stop Streaming Events

In [6]:
sink_player.stop()
sink_purchase_weapon.stop()
sink_purchase_shield.stop()
sink_dig_for_gold.stop()
sink_successful_attack.stop()
sink_failed_attack.stop()

### Step 4: Read Events From HDFS and Create External Hive Tables

#### 4.1 Read parquet files using Spark and Register Tables

In [7]:
#player_events
transformed_player_events = spark.read.parquet('/tmp/player')
transformed_player_events.registerTempTable('player')

#purchase_weapon_events
transformed_purchase_weapon_events = spark.read.parquet('/tmp/purchase_weapon')
transformed_purchase_weapon_events.registerTempTable('purchase_weapon')

#purchase_shield_events
transformed_purchase_shield_events = spark.read.parquet('/tmp/purchase_shield')
transformed_purchase_shield_events.registerTempTable('purchase_shield')

#dig_for_gold_events
transformed_dig_for_gold_events = spark.read.parquet('/tmp/dig_for_gold')
transformed_dig_for_gold_events.registerTempTable('dig_for_gold')

#successful_attack_events
transformed_successful_attack_events = spark.read.parquet('/tmp/successful_attack')
transformed_successful_attack_events.registerTempTable('successful_attack')

#failed_attack_events
transformed_failed_attack_events = spark.read.parquet('/tmp/failed_attack')
transformed_failed_attack_events.registerTempTable('failed_attack')

#### 4.2 Copy the dataframe to Pandas and Display Data

In [8]:
transformed_player_events_pd = transformed_player_events.toPandas()
transformed_player_events_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,username
0,"{""username"": ""user50"", ""Host"": ""user1.comcast....",2021-08-06 08:30:09.723,*/*,user1.comcast.com,ApacheBench/2.3,initialize_player,user50
1,"{""username"": ""user25"", ""Host"": ""user1.comcast....",2021-08-06 08:30:12.626,*/*,user1.comcast.com,ApacheBench/2.3,initialize_player,user25
2,"{""username"": ""user34"", ""Host"": ""user1.comcast....",2021-08-06 08:30:13.413,*/*,user1.comcast.com,ApacheBench/2.3,initialize_player,user34
3,"{""username"": ""user27"", ""Host"": ""user1.comcast....",2021-08-06 08:30:21.335,*/*,user1.comcast.com,ApacheBench/2.3,initialize_player,user27
4,"{""username"": ""user26"", ""Host"": ""user1.comcast....",2021-08-06 08:30:25.631,*/*,user1.comcast.com,ApacheBench/2.3,initialize_player,user26


In [9]:
transformed_purchase_weapon_events_pd = transformed_purchase_weapon_events.toPandas()
transformed_purchase_weapon_events_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,username,weapon,wallet_before,wallet_after
0,"{""username"": ""user34"", ""event_type"": ""purchase...",2021-08-06 08:36:07.011,*/*,user1.comcast.com,ApacheBench/2.3,purchase_weapon,user34,Grenade,268,253
1,"{""username"": ""user48"", ""event_type"": ""purchase...",2021-08-06 08:33:26.236,*/*,user1.comcast.com,ApacheBench/2.3,purchase_weapon,user48,Grenade,236,221
2,"{""username"": ""user14"", ""event_type"": ""purchase...",2021-08-06 08:33:31.202,*/*,user1.comcast.com,ApacheBench/2.3,purchase_weapon,user14,Bazooka,259,159
3,"{""username"": ""user28"", ""event_type"": ""purchase...",2021-08-06 08:34:10.994,*/*,user1.comcast.com,ApacheBench/2.3,purchase_weapon,user28,Grenade,231,216
4,"{""username"": ""user27"", ""event_type"": ""purchase...",2021-08-06 08:34:17.833,*/*,user1.comcast.com,ApacheBench/2.3,purchase_weapon,user27,Grenade,155,140


In [10]:
transformed_purchase_shield_events_pd = transformed_purchase_shield_events.toPandas()
transformed_purchase_shield_events_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,username,wallet_before,wallet_after
0,"{""username"": ""user14"", ""event_type"": ""purchase...",2021-08-06 08:36:47.201,*/*,user1.comcast.com,ApacheBench/2.3,purchase_shield,user14,159,139
1,"{""username"": ""user25"", ""event_type"": ""purchase...",2021-08-06 08:38:17.795,*/*,user1.comcast.com,ApacheBench/2.3,purchase_shield,user25,159,139
2,"{""username"": ""user13"", ""event_type"": ""purchase...",2021-08-06 08:35:36.806,*/*,user1.comcast.com,ApacheBench/2.3,purchase_shield,user13,131,111
3,"{""username"": ""user48"", ""event_type"": ""purchase...",2021-08-06 08:31:39.002,*/*,user1.comcast.com,ApacheBench/2.3,purchase_shield,user48,162,142
4,"{""username"": ""user27"", ""event_type"": ""purchase...",2021-08-06 08:33:05.628,*/*,user1.comcast.com,ApacheBench/2.3,purchase_shield,user27,195,175


In [11]:
transformed_dig_for_gold_events_pd = transformed_dig_for_gold_events.toPandas()
transformed_dig_for_gold_events_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,username,gold_found,wallet_before,wallet_after
0,"{""username"": ""user26"", ""event_type"": ""dig_for_...",2021-08-06 08:37:00.379,*/*,user1.comcast.com,ApacheBench/2.3,dig_for_gold,user26,14,156,165
1,"{""username"": ""user26"", ""event_type"": ""dig_for_...",2021-08-06 08:37:00.387,*/*,user1.comcast.com,ApacheBench/2.3,dig_for_gold,user26,6,165,166
2,"{""username"": ""user26"", ""event_type"": ""dig_for_...",2021-08-06 08:37:00.4,*/*,user1.comcast.com,ApacheBench/2.3,dig_for_gold,user26,88,166,249
3,"{""username"": ""user26"", ""event_type"": ""dig_for_...",2021-08-06 08:37:00.406,*/*,user1.comcast.com,ApacheBench/2.3,dig_for_gold,user26,25,249,269
4,"{""username"": ""user26"", ""event_type"": ""dig_for_...",2021-08-06 08:37:00.413,*/*,user1.comcast.com,ApacheBench/2.3,dig_for_gold,user26,141,269,405


In [12]:
transformed_successful_attack_pd = transformed_successful_attack_events.toPandas()
transformed_successful_attack_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,attacker,defender,weapon_used,defender_has_shield,defender_health_before,defender_health_after,defender_killed
0,"{""event_type"": ""successful_attack"", ""weapon_us...",2021-08-06 08:34:03.834,*/*,user1.comcast.com,ApacheBench/2.3,successful_attack,user50,user47,Knife,False,20,18,False
1,"{""event_type"": ""successful_attack"", ""weapon_us...",2021-08-06 08:34:05.086,*/*,user1.comcast.com,ApacheBench/2.3,successful_attack,user39,user47,Stick,False,18,18,False
2,"{""event_type"": ""successful_attack"", ""weapon_us...",2021-08-06 08:35:33.649,*/*,user1.comcast.com,ApacheBench/2.3,successful_attack,user48,user36,Grenade,False,20,15,False
3,"{""event_type"": ""successful_attack"", ""weapon_us...",2021-08-06 08:34:31.932,*/*,user1.comcast.com,ApacheBench/2.3,successful_attack,user48,user19,Grenade,False,11,6,False
4,"{""event_type"": ""successful_attack"", ""weapon_us...",2021-08-06 08:37:29.812,*/*,user1.comcast.com,ApacheBench/2.3,successful_attack,user34,user21,Grenade,True,20,20,False


In [15]:
transformed_failed_attack_pd = transformed_failed_attack_events.toPandas()
transformed_failed_attack_pd.head()

,raw_event,timestamp,Accept,Host,User-Agent,event_type,attacker,defender,weapon_used
0,"{""event_type"": ""failed_attack"", ""weapon_used"":...",2021-08-06 08:34:25.297,*/*,user1.comcast.com,ApacheBench/2.3,failed_attack,user21,user30,Grenade
1,"{""event_type"": ""failed_attack"", ""weapon_used"":...",2021-08-06 08:36:18.075,*/*,user1.comcast.com,ApacheBench/2.3,failed_attack,user48,user50,Grenade
2,"{""event_type"": ""failed_attack"", ""weapon_used"":...",2021-08-06 08:37:36.112,*/*,user1.comcast.com,ApacheBench/2.3,failed_attack,user34,user31,Grenade
3,"{""event_type"": ""failed_attack"", ""weapon_used"":...",2021-08-06 08:37:39.101,*/*,user1.comcast.com,ApacheBench/2.3,failed_attack,user48,user20,Grenade
4,"{""event_type"": ""failed_attack"", ""weapon_used"":...",2021-08-06 08:38:21.833,*/*,user1.comcast.com,ApacheBench/2.3,failed_attack,user34,user10,Grenade


#### 4.3 Create schema-on-read (external tables) for the parquet files in HDFS

In [8]:
#player_events
player_query = "create external table player_events stored as parquet location '/tmp/player_events' as select * from player"
spark.sql(player_query)

#purchase_weapon_events
purchase_weapon_query = "create external table purchase_weapon_events stored as parquet location '/tmp/purchase_weapon_events' as select * from purchase_weapon"
spark.sql(purchase_weapon_query)

#purchase_shield_events
purchase_shield_query = "create external table purchase_shield_events stored as parquet location '/tmp/purchase_shield_events' as select * from purchase_shield"
spark.sql(purchase_shield_query)

#dig_for_gold_events
dig_for_gold_query = "create external table dig_for_gold_events stored as parquet location '/tmp/dig_for_gold_events' as select * from dig_for_gold"
spark.sql(dig_for_gold_query)

#successful_attack_events
successful_attack_query = "create external table successful_attack_events stored as parquet location '/tmp/successful_attack_events' as select * from successful_attack"
spark.sql(successful_attack_query)

#failed_attack_events
failed_attack_query = "create external table failed_attack_events stored as parquet location '/tmp/failed_attack_events' as select * from failed_attack"
spark.sql(failed_attack_query)

DataFrame[]

## Data Analysis using Presto

    1. Run Presto on container
 ```console
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default


 ```
    2. Check and Describe the tables on Presto CLI

```console
show tables;
```
```console
          Table           
--------------------------
 dig_for_gold_events      
 failed_attack_events     
 player_events            
 purchase_shield_events   
 purchase_weapon_events   
 successful_attack_events 
(6 rows)
```


```console
describe player_events;

```
```console
   Column   |  Type   | Comment 
------------+---------+---------
 raw_event  | varchar |         
 timestamp  | varchar |         
 accept     | varchar |         
 host       | varchar |         
 user-agent | varchar |         
 event_type | varchar |         
 username   | varchar |         
(7 rows)

```
```console
describe failed_attack_events;
```
```console
   Column    |  Type   | Comment 
-------------+---------+---------
 raw_event   | varchar |         
 timestamp   | varchar |         
 accept      | varchar |         
 host        | varchar |         
 user-agent  | varchar |         
 event_type  | varchar |         
 attacker    | varchar |         
 defender    | varchar |         
 weapon_used | varchar |         
(9 rows)
```

    3. Answer business analysis questions using presto queries
        - Our business analysis questions dealt with how a randomized gameplay with randomized requests would result in different aspects of our game. For example, we wanted to see how many players were created, how much gold was won, what weapons were used etc. There are many different analysis questions that we could have pursued, but we thought the below 9 would be interesting to see considering the entire gameplay was random.

### Q1: How many distinct players have been created

Query:
```sql
SELECT Count(DISTINCT( username )) AS number_of_players
FROM   player_events;
```

Query Result:
```console
 number_of_players 
-------------------
                43 
(1 row)

Query 20210806_084826_00003_ncxcs, FINISHED, 1 node
Splits: 16 total, 14 done (87.50%)
0:00 [43 rows, 6.05KB] [112 rows/s, 15.8KB/s]
```

**Answer**: 43 distinct players have been created (there won't be any duplicates on username because our game does not allow repeated username to be created using Redis state tracking)

### Q2: How often is each weapon purchased?

Query:
```sql
SELECT weapon,
       Count(weapon) AS weapon_purchase_count
FROM   purchase_weapon_events
GROUP  BY weapon
ORDER  BY weapon_purchase_count DESC;
```

Query Result:
```console
 weapon  | weapon_purchase_count 
---------+-----------------------
 Stick   |                    24 
 Knife   |                    22 
 Gun     |                    17 
 Bazooka |                    15 
 Grenade |                    12 
 Sword   |                    10 
(6 rows)

Query 20210806_085120_00004_ncxcs, FINISHED, 1 node
Splits: 15 total, 10 done (66.67%)
0:01 [100 rows, 17.8KB] [161 rows/s, 28.7KB/s]
```

**Answer**: A Stick seems to have been purchased most often, 24 times, followed by a Knife and a Gun. It appears that a Nuke was never successfully purchased in the game till now.

### Q3: What is the average wallet size after the purchase of a shield?

Query:
```sql
SELECT Avg(wallet_after) AS wallet_after_purchasing_shield
FROM   purchase_shield_events; 
```

Query Result:
```
 wallet_after_purchasing_shield 
--------------------------------
             104.39285714285714 
(1 row)

Query 20210806_085438_00006_ncxcs, FINISHED, 1 node
Splits: 15 total, 0 done (0.00%)
0:00 [0 rows, 0B] [0 rows/s, 0B/s]
```

**Answer**: It seems that the average wallet of a user after purchasing a shield was 104.4.

### Q4: How many players purchased a shield more than once?

Query:
```sql
SELECT Count(username) AS players_shield_purchases_more_than_one
FROM   (SELECT username,
               Count(username) AS shield_purchases
        FROM   purchase_shield_events
        GROUP  BY( username ))
        WHERE  shield_purchases > 1; 
```

Query Result:
```console
 players_shield_purchases_more_than_one 
----------------------------------------
                                     15 
(1 row)

Query 20210806_085637_00007_ncxcs, FINISHED, 1 node
Splits: 16 total, 0 done (0.00%)
0:00 [0 rows, 0B] [0 rows/s, 0B/s]
```
**Answer**: 15 users purchased a shield more than once. Given the game design, this was a redundent purchase.

### Q5: What is the max, min, and average of gold found by a user?

Query:
```sql
SELECT Max(gold_found) AS max,
       Min(gold_found) AS min,
       Avg(gold_found) AS avg
FROM   dig_for_gold_events; 
```

Query Result:
```
 max | min |        avg         
-----+-----+--------------------
 141 |   0 | 26.595744680851062 
(1 row)

Query 20210806_085800_00008_ncxcs, FINISHED, 1 node
Splits: 15 total, 15 done (100.00%)
0:01 [235 rows, 19.6KB] [442 rows/s, 36.8KB/s]
```

**Answer**: The maximum a user won by digging for gold was 141, while the least was 0. The average of 26.6 is pretty close to the game design expected value of 25. This is good, as it means that the simulation is large enough to represent the long-run play of the game

### Q6: What weapon fails the most often?


Query:
```sql
SELECT weapon_used,
       Count(weapon_used) AS count_weapon_used
FROM   failed_attack_events
GROUP  BY weapon_used
ORDER  BY count_weapon_used DESC; 
```

Query Result:
```
 weapon_used | count_weapon_used 
-------------+-------------------
 Grenade     |                 7 
 Sword       |                 7 
 Bazooka     |                 5 
 Knife       |                 3 
 Gun         |                 3 
(5 rows)

Query 20210806_091342_00020_ncxcs, FINISHED, 1 node
Splits: 15 total, 0 done (0.00%)
0:00 [0 rows, 0B] [0 rows/s, 0B/s]
```

**Answer**: A Grenade seems to have failed the most times, followed by a Sword and then a Bazooka.

### Q7: Which weapon kills the most often?

Query:
```sql
SELECT weapon_used,
       Count(weapon_used) AS count_weapon_used_to_kill
FROM   successful_attack_events
WHERE  defender_killed = True
GROUP  BY weapon_used
ORDER by count_weapon_used_to_kill DESC; 
```

Query Result:
```console
 weapon_used | count_weapon_used_to_kill 
-------------+---------------------------
 Bazooka     |                         4 
 Gun         |                         1 
(2 rows)

Query 20210806_090314_00014_ncxcs, FINISHED, 1 node
Splits: 15 total, 0 done (0.00%)
0:00 [0 rows, 0B] [0 rows/s, 0B/s]
```

**Answer**: It seems like a Bazooka was used most often to kill, followed by a Gun. This is expected, as those weapons are the ones purchased that do the most damage.

### Q8: How many players have been killed in the game so far?

Query:
```sql
SELECT Count(defender_killed) AS count_defender_killed
FROM   successful_attack_events
WHERE  defender_killed = True;
```

Query Result:
```console
 count_defender_killed 
-----------------------
                     5 
(1 row)

Query 20210806_090518_00017_ncxcs, FINISHED, 1 node
Splits: 14 total, 1 done (7.14%)
0:00 [64 rows, 12.2KB] [178 rows/s, 33.9KB/s]
```

**Answer**: 5 of the 43 players seem to have been killed so far. This is lower than we were expecting given how long we have run the simulation. Perhaps the weapons need to have higher damage, or probability of success to make the game play more interesting.

### Q9: How many defenders had a shield during an attack?

Query:
```sql
SELECT Count(defender) AS count_has_shield
FROM   successful_attack_events
WHERE  defender_has_shield = True; 
```

Query Result:
```console
 count_has_shield 
------------------
               62 
(1 row)

Query 20210806_090752_00018_ncxcs, FINISHED, 1 node
Splits: 14 total, 0 done (0.00%)
0:00 [0 rows, 0B] [0 rows/s, 0B/s]
```
**Answer**: 62 defenders (47% as total successful attacks were 132) in the game had a shield during an attack. This is a very high number, and might be another reason why not many players died. Perhaps we could increase the cost of a shield to make it a more rare event for a defender to have a shield.

### Step 5: Bring Down Docker Cluster

```console
docker-compose down
```